# Bouton density

In this notebook you will calculate the bouton density for each morphological type (m-type) and compare the result with available experimental data.

Bouton density is defined as the number of boutons per axon length (generally per 100 $\mu$m or per $\mu$m).

---

Import some python packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn
import json
from pathlib import Path

# package to work with the circuit
from bluepysnap import Circuit
from bluepysnap.bbp import Cell
import neurom as nm

Reading and preparing the data

In [ ]:
circuit_path = '/home/data-bbp/20191017/circuit_config.json'
circuit = Circuit(circuit_path)
cells = circuit.nodes["hippocampus_neurons"]
conn = circuit.edges["hippocampus_neurons__hippocampus_neurons__chemical"]

### Analysis

Initialize table to store the results.

In [ ]:
mtypes = cells.property_values(Cell.MTYPE)
df = pd.DataFrame(index=mtypes, columns=['mean', 'std'])

Define function to compute the bouton density in a sample of cells.

In [ ]:
def sample_bouton_density(circuit, sample, group=None, synapses_per_bouton=1.0):
    def _calc_bouton_density(gid, synapses_per_bouton):
        synapse_count = len(conn.efferent_edges(gid))
        axon_length = nm.get(
            'neurite_lengths', cells.morph.get(int(gid), False), neurite_type=nm.AXON
        )[0]
        return (1.0 * synapse_count / synapses_per_bouton) / axon_length

    gids = cells.ids(group)
    print(f" number of cells {len(gids)}")
    if len(gids) > sample:
        gids = np.random.choice(gids, size=sample, replace=False)
    return np.array([_calc_bouton_density(gid, synapses_per_bouton) for gid in gids])

Iterate over all the m-type, compute the bouton density, and store the result.

In [ ]:
sample = 500

for i, mtype in enumerate(mtypes, 1):
    print(f"{mtype}, {i}/{len(mtypes)} mtypes")
    data = sample_bouton_density(circuit, sample, group={Cell.MTYPE: mtype, Cell.REGION: {'$regex': 'mc2.*'}}, synapses_per_bouton=1.15) 
    if len(data) == 0:
        print(f" {mtype}: no button to sample mean and std will be set to 0")
    df.loc[mtype]['mean'] = data.mean() if len(data) != 0 else 0
    df.loc[mtype]['std'] = data.std() if len(data) != 0 else 0

df.head(len(mtypes))

Visualize the result.

In [ ]:
fig, ax = plt.subplots()

labels = mtypes
ind = np.arange(len(labels))
width = 0.75

s = ax.bar(ind, df['mean'], width, yerr=df.loc[mtype]['std'])

ax.set_xlabel('mtype')
ax.set_ylabel('density (um^-1)')
ax.set_title('Bouton density')
ax.set_xticks(ind)
ax.set_xticklabels(labels, rotation='vertical')

fig.tight_layout()

fig.show()

### Validation

In some cases, experimental data is available to validate the bouton density in the model.

---

Load experimental datapoints previously stored in a TSV file

In [ ]:
bio_path = '/home/data-bbp/20191017/bioname/bouton_density_20190131.tsv'

In [ ]:
data = pd.read_csv(bio_path, names=['mtype', 'bio_mean', 'bio_std'], skiprows=2, usecols=[0,1,2], delim_whitespace=True)
data.head()

Merge model results and experimental datapoint in a single dataframe to facilitate further data manipulation.

In [ ]:
selected = data['mtype'].values
data['mod_mean'] = df.loc[selected]['mean'].values
data['mod_std'] = df.loc[selected]['std'].values
data.head()

Visualize model results versus experimental data. Dashed line represents a perfect match between model and experiment.

In [ ]:
plt.close('all')

x = data['mod_mean'].values
y = data['bio_mean'].values
# remove nan value from the array
# l = np.linspace(0, max(x[~np.isnan(x)].max(), y.max()), 50)
l = np.linspace(0, max(x.max(), y.max()), 50)
fig, ax = plt.subplots()
fig.suptitle('Bouton density')
ax.plot(x, y, 'o')
ax.errorbar(x, y, xerr=data['mod_std'].values, yerr=data['bio_std'].values, fmt='o', ecolor='g', capthick=2)
ax.plot(l, l, 'k--')
ax.set_xlabel('Model (um^-1)')
ax.set_ylabel('Experiment (um^-1)')

fig.show()

In [ ]:
# Work here

### Exercise #1
Calculate the average bouton density in an interneuron and a neuron (either excitatory or inhibitory). Store the answers in _ans\_1a_ for the interneuron, and in _ans\_1b_ for the neuron. Each answer is a list with two elements: the mean value and the standard deviation of the bouton density.

__Hint__: to query GIDs of interneurons use Cell.SYNAPSE_CLASS. When you query all the interneurons, you do not need any filters.

### Exercise #2
Calculate the average bouton density in a neuron (either excitatory or inhibitory) in column 0 (zero). Similarly to the previous exercise, store the mean and the standard deviation in a list and assign it to _ans\_2a_. Are the values lower, bigger or the same of the bouton densities calculated in the central column? Store 'lower', 'bigger', or 'same' in _ans\_2b_. As for ungraded exercise, can you explain why you found or not found a difference?

In [ ]:
# This is to generate the answers to paste in the submission box below.
# After you defined the variables with your answers, run this cell and the next cell, and copy-paste the output into the box below
print(json.dumps(dict([("ans_1a", ans_1a),
                       ("ans_1b", ans_1b),
                       ("ans_2a", ans_2a),
                       ("ans_2b", ans_2b)])))

In [ ]:
import single_cell_mooc_client as sc_mc
s = sc_mc.Submission()